In [1]:
mkdir github


In [2]:
cd github 

/home/aryan/SOC/Final_task/github


In [4]:
!git clone https://github.com/mwydmuch/ViZDoom


Cloning into 'ViZDoom'...
remote: Enumerating objects: 19354, done.
remote: Counting objects: 100% (2526/2526), done.
remote: Compressing objects: 100% (905/905), done.
remote: Total 19354 (delta 1806), reused 1651 (delta 1617), pack-reused 16828 (from 2)
Receiving objects: 100% (19354/19354), 73.98 MiB | 4.00 MiB/s, done.
Resolving deltas: 100% (12405/12405), done.


In [5]:
!python -m vizdoom.scenarios.basic

/home/aryan/miniconda3/envs/doom_env/bin/python: No module named vizdoom.scenarios.basic


In [6]:
!cd github

/bin/bash: line 1: cd: github: No such file or directory


In [7]:
!ls

ViZDoom


In [8]:
!python -m scenarios.basic

/home/aryan/miniconda3/envs/doom_env/bin/python: Error while finding module specification for 'scenarios.basic' (ModuleNotFoundError: No module named 'scenarios')


In [ ]:
# from vizdoom import *
# from vizdoom import DoomGame
# import random
# import time

# import numpy as np


In [2]:
game =DoomGame()
game.load_config("github/ViZDoom/scenarios/basic.cfg")
game.init()

In [47]:
action_space=np.identity(3,dtype=int)

In [28]:
game.new_episode()
game.make_action(random.choice(action_space))

-1.0

In [44]:
game.make_action(random.choice(action_space))

-1.0

In [48]:
for i in range(5):
    game.new_episode()
    while not game.is_episode_finished():
        state=game.get_state()
        img= state.screen_buffer
        info=state.game_variables
        reward=game.make_action(random.choice(action_space))
        print(i,reward)
        time.sleep(0.05)
    print(game.get_total_reward())



0 -1.0
0 -1.0
0 -1.0
0 -1.0
0 -1.0
0 -1.0
0 -6.0
0 -1.0
0 -1.0
0 -1.0
0 -1.0
0 -1.0
0 -1.0
0 -1.0
0 -1.0
0 -1.0
0 -1.0
0 -1.0
0 -1.0
0 -1.0
0 -1.0
0 -1.0
0 -1.0
0 -1.0
0 -1.0
0 -1.0
0 -1.0
0 -6.0
0 -1.0
0 -1.0
0 -1.0
0 -1.0
0 -1.0
0 -1.0
0 -1.0
0 -1.0
0 -1.0
0 -1.0
0 -1.0
0 -1.0
0 -1.0
0 100.0
0 -1.0
48.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -6.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -6.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -6.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -6.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -6.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0
1 -1.0


# Creating GYM wrapper


In [1]:
from vizdoom import *
from vizdoom import DoomGame

import numpy as np
import matplotlib.pyplot as plt

import gymnasium as gym

from gymnasium import Env


from gymnasium.spaces import Box,Discrete

import cv2

import os

from stable_baselines3.common.callbacks import BaseCallback

from stable_baselines3.common import env_checker


In [3]:
CHECK_POINT_DIR="./train/Centre"
LOG_DIR="./log/Centre"

In [5]:

class TrainAndLoggingCallback(BaseCallback):
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)
            if self.verbose:
                print(f" Model checkpoint saved at step {self.n_calls} → {model_path}")
        return True

In [6]:
callback= TrainAndLoggingCallback(check_freq=10000,save_path=CHECK_POINT_DIR)

In [ ]:
class VizDoomEnv(Env):
    def __init__ (self,render=False,config="github/ViZDoom/scenarios/basic.cfg",tics=2):
        super().__init__()
        self.game=DoomGame()
        self.game.load_config(config)
        self.game.set_window_visible(render)

        self.game.init()
        self.tics=tics

        self.observation_space=Box(low=0,high=255,shape=(100,160,1), dtype=np.uint8)
        self.action_space=Discrete(3)

    def step(self, action):
        def step(self, action):
            actions = np.identity(3, dtype=np.uint8)
            reward = self.game.make_action(actions[action],self.tics)

            terminated = self.game.is_episode_finished()
            truncated = False  # You can implement time-limit logic if needed

            if self.game.get_state():
                state = self.game.get_state().screen_buffer
                state = self.greyscale(state)
                ammo = self.game.get_state().game_variables[0]
                info = {"ammo": ammo}
            else:
                state = np.zeros(self.observation_space.shape, dtype=np.uint8)
                info = {"ammo": 0}

            return state, reward, terminated, truncated, info

    def render(self):
        pass
    def greyscale(self,observation):
        grey= cv2.cvtColor(np.moveaxis(observation,0,-1),cv2.COLOR_BGR2GRAY) #move axis to reoder the channels to (240,320,3)
        resize=cv2.resize(grey,(160,100),interpolation=cv2.INTER_CUBIC)
        state=np.reshape(resize,(100,160,1))
        return state
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)  # Let Gymnasium handle seeding
        if seed is not None:
            np.random.seed(seed)  # Optional, for reproducibility

        self.game.new_episode()
        observation = self.greyscale(self.game.get_state().screen_buffer)

        info = {}  # You can add useful debug info here if needed
        return observation, info
    def close(self):
        self.game.close()

In [8]:
env=VizDoomEnv(render=False)
#env.step(0)


In [42]:

state=env.reset()
state.shape

(100, 160, 1)

In [11]:
env.close()

# Moving on

In [ ]:
!pip install stable-baselines3[extra]


In [9]:
env_checker.check_env(env)